# Basics

- bson = binary json. MongoDB uses this to store data records bc BSON has a much larger list of support data types than JSON

- `mongosh` creates a MongoDB shell that is in JavaScript

- You must start a server before connecting to it in VSCode

- run `mongod` to start server

- `cls`, `Ctrl+L`, or `console.clear()` clears the mongodb shell

## Shell Commands

- `use [db name]` switches to that database
- You can create documents:
  - `doc = {"title": "Tacos", "desc":"Yummy Tacos", "cook_time":20};`
- `db.tacos.insertOne(doc)` inserts that document into the tacos collection
- `show dbs;`
- `show collections;`
- `db.getName();` get current db
- `db.cool_newStuff.insertOne({});` can create collections this way too
- `db.cool.newStuff.insertOne({})` creates a sub collection new stuff in cool collection
- `cap collection` tells mongodb to store a max num of docs and deletes oldest to make room for new one
- `db.dropDatabase()` deletes the current db


# Mongosh Tips

#### Help()

Gives you some docs and an example for a function that you need help with.

Example:

`cooker> db.recipes.createIndex.help()`

**returns:**

  db.coll.createIndex({ category: 1 }, { name: 'index-1' }):

  Creates one index on a collection

  For more information on usage: https://docs.mongodb.com/manual/reference/method/db.collection.createIndex

#### It's a JS shell

You can define variables and use js functions:

`const results = db.recipes.find({}, {title:1, prep_time:1, _id:0}).toArray()`

`console.table(results)`

**If not using mongosh **add `.pretty()` at end for formatting after each query 

#### Autocomplete

For example if you want to get all functions that start with gt, then hit tab

db.recipes.find({cook_time:{ $gt [tab]

**It doesn't work for me, so I'll have to fix it, but it's supposed to return $gt and $gte**



# Commands

## find()

`cooker> db.<collection_name>.find()`

- creates a new collection if it doesn't exist
- find() returns everything inside that collection
- An **_id** field is inserted into each document

### Conditionals with find():

`cooker> db.recipes.find({title: "Tacos", title: 1
})`

- `title:1` Only return the title field (and _ids)
- Matches items with title Tacos

`cooker> db.recipes.find({title: "Tacos", title: 0
})`

- `title:0` exclude the title field when returned

`cooker> db.recipes.find({},
{title: 1
})`

- Returns all items' titles

#### Regex

We are running a JavaScript shell in MongoDB, so we can use regex.

`db.recipes.find({title: { $regex: /taco/i
  }
})`

- `$regex` to use regex
- `/taco` match taco in title
- `/i` case insensitive

Returns:

\[
  \{
    _id: ObjectId("5e6fd805fa98021236426a24"),
    title: 'Chicken Soft Tacos'
  \},  
  \{ _id: ObjectId("5e5e9c470d33e9e8e3891b35"), title: 'Tacos' \}
\]

Collections are like tables in a relational database., 

not all the documents need the same schema or structure

a document is like a row in a table in a relation db

documents can be arrays of strings or objects, or other types

## Chaining

db returns a cursor when you query.
when you run the find() command, it's sent to the server but doesn't return results immediately. waits to get the results back. bc of this, we can add extra commands. this is **chaining**

`db.recipes.find({}, {title:1}).skip(2).limit(3).sort(title:1)`

- `limit([max num of docs to return])` limits results returned
- `count()` number of docs
- `sort([field to sort by] : [1 or -1])` 1 ascending, -1 descending
- `skip([num of doc to skip])` skips from the top in alphabetical order

## Operations

- $gt
- $lt
- {$lte: value}
- {$or : [array of filters]}

`cooker> db.recipes.find({cook_time:{$lte:30}, prep_time:{$lte:10}},   {title:1, cook_time:1, prep_time:1}).limit(5)`  
[  
  {
    _id: ObjectId("5e6fd805fa98021236426a24"),
    title: 'Chicken Soft Tacos',
    cook_time: 19,
    prep_time: 10
  },  
  {
    _id: ObjectId("5e877cba20a4f574c0aa56da"),
    title: 'Pancakes',
    cook_time: 10,
    prep_time: 10
  },  
  {
    _id: ObjectId("5e5e9c470d33e9e8e3891b35"),
    title: 'Tacos',
    cook_time: 20,
    prep_time: 10
  }  
]

`cooker> db.recipes.find({$or:[{cook_time:{$lte:30}}, {prep_time:{$lte:10}}]}, {title:1, cook_time:1, prep_time:1}).limit(5)`  
[  
  {
    _id: ObjectId("5e6fd805fa98021236426a24"),
    title: 'Chicken Soft Tacos',
    cook_time: 19,
    prep_time: 10
  },  
  {
    _id: ObjectId("5e877cba20a4f574c0aa56da"),
    title: 'Pancakes',
    cook_time: 10,
    prep_time: 10
  },  
  {
    _id: ObjectId("5e5e9c470d33e9e8e3891b35"),
    title: 'Tacos',
    cook_time: 20,
    prep_time: 10
  },  
  {
    _id: ObjectId("5e878f5220a4f574c0aa56db"),
    title: 'Maple Smoked Salmon',
    cook_time: 20,
    prep_time: 15
  }  
]

## More Conditionals

❌ `cooker> db.recipes.find({tags:["easy", "quick"]});`
> mongodb does an exact match on tags with easy and quick and returns nothing

To avoid an exact match:
- {$all : {attribute: [filters]}} - AND for each item in the filter
- {$in : {attribute: [filters]}} - OR

✅ `cooker> db.recipes.find({tags:{$all: ["easy", "quick"]}}, {title:1, tags:1}) ` 
[
  {
    _id: ObjectId("5e6fd805fa98021236426a24"),
    title: 'Chicken Soft Tacos',
    tags: [ 'mexican', 'quick', 'easy', 'chicken' ]
  },  
  {
    _id: ObjectId("5e5e9c470d33e9e8e3891b35"),
    title: 'Tacos',
    tags: [ 'mexican', 'quick', 'easy', 'ground beef' ]
  }  
]

✅ `cooker> db.recipes.find({tags:{$in: ["easy", "quick"]}}, {title:1, tags:1})`  
[
  {
    _id: ObjectId("5e6fd805fa98021236426a24"),
    title: 'Chicken Soft Tacos',
    tags: [ 'mexican', 'quick', 'easy', 'chicken' ]
  },  
  {
    _id: ObjectId("5edf1d313260aab97ea0d589"),
    title: 'Zucchini Brownies',
    tags: [ 'sweets', 'easy' ]
  },  
  {
    _id: ObjectId("5e87856d07beb474c074c5ca"),
    title: 'Brown Sugar Meatloaf',
    tags: [ 'ground beef', 'family meal', 'easy' ]
  },  
  {
    _id: ObjectId("5e5e9c470d33e9e8e3891b35"),
    title: 'Tacos',
    tags: [ 'mexican', 'quick', 'easy', 'ground beef' ]
  }  
]

## Dot notation

dot notation is used to go inside a db and used when you don't want an exact match. Use quotes around it.

❌ `db.recipes.find({ingredients:{"name": "egg"}})`
> Doesn't return anything, because mongodb is looking for an exact match!



✅ `cooker> db.recipes.find({"ingredients.name":"egg"}, {title:1, "ingredients.name":1})`  
[
  {
    _id: ObjectId("5edf1d313260aab97ea0d589"),
    title: 'Zucchini Brownies',
    ingredients: [
      { name: 'butter' },    
      { name: 'egg' },  
      { name: 'plain yogurt' },
      { name: 'shredded zucchini' },
      { name: 'chocolate chips (semisweet)' },
      { name: 'creamy peanut butter' },
      { name: 'brownie mix' }
    ]
  },  
  {
    _id: ObjectId("5e877cba20a4f574c0aa56da"),
    title: 'Pancakes',
    ingredients: [
      { name: 'milk' },
      { name: 'white vinegar' },
      { name: 'baking soda' },
      { name: 'salt' },
      { name: 'granulated sugar' },
      { name: 'baking powder' },
      { name: 'all-purpose flour' },  
      { name: 'egg' },  
      { name: 'butter' }
    ]
  }  
]

## updateOne()

`cooker> db.examples.find({}, {"title":1});`
[  
  { _id: ObjectId("5ee69e393260aab97ea0d58e"), title: 'Delete me' },  
  **{ _id: ObjectId("5ee69d943260aab97ea0d58d"), title: 'Pizza' },**  
  { _id: ObjectId("5e5e9c470d33e9e8e3891b35"), title: 'Tacos' }  
]

### Set

`db.examples.updateOne({"title": "Pizza"}, {$set:{title:"Thin crust pizza"}})`

[...,  
  {
    _id: ObjectId("5ee69d943260aab97ea0d58d"),
    **title: 'Thin crust pizza'**
  },  
...]

### Insert

`db.examples.updateOne({"title": "Thin crust pizza"}, {$set:{vegan:false}});`

[...,  
  {
    _id: ObjectId("5ee69d943260aab97ea0d58d"),
    title: 'Thin crust pizza',
    **vegan: false**
  },  
...]

### Unset

` db.examples.updateOne({"title": "Thin crust pizza"}, {$unset:{vegan:1}})`

[...,  
  {
    _id: ObjectId("5ee69d943260aab97ea0d58d"),
    title: 'Thin crust pizza'
  },  
...]

### Increment Operation

**Eventual Consistency:**

Guarantee the field will always increase (or decrease) by the proper amount. It's atomic! No locks or transactions

**{$inc:  {  [field]:  [amount to increase by]}}**

`cooker> db.examples.find({"title":"Tacos"}, {title:1, likes_count:1})`  
[
  {
    _id: ObjectId("5e5e9c470d33e9e8e3891b35"),
    title: 'Tacos',
    **likes_count: 2**  
  }
]  

`db.examples.updateOne({"title": "Tacos"}, {$inc:{likes_count:1}})`

[
  {
    _id: ObjectId("5e5e9c470d33e9e8e3891b35"),
    title: 'Tacos',
    **likes_count: 3**  
  }
]  

Can decrease too:  
`db.examples.updateOne({"title": "Tacos"}, {$inc:{likes_count:-1}})`

[
  {
    _id: ObjectId("5e5e9c470d33e9e8e3891b35"),
    title: 'Tacos',
    **likes_count: 2**  
  }
]  

## Array Operations

`cooker> db.examples.find({"title":"Tacos"}, {title:1,likes:1})`  
[
  {
    _id: ObjectId("5e5e9c470d33e9e8e3891b35"),
    title: 'Tacos',
    **likes: [ 1, 415 ]**
  }
]

### Push

`db.examples.updateOne({"title":"Tacos"}, {$push: {"likes":60}})`

[
  {
    _id: ObjectId("5e5e9c470d33e9e8e3891b35"),
    title: 'Tacos',
    **likes: [ 1, 415, 60 ]**
  }
]

### Pull

`db.examples.updateOne({"title":"Tacos"}, {$pull: {"likes":60}})`

[
  {
    _id: ObjectId("5e5e9c470d33e9e8e3891b35"),
    title: 'Tacos',
    **likes: [ 1, 415 ]**
  }
]

## Deleting Documents

`cooker> db.examples.find({}, {title:1})`   
[
  **{ _id: ObjectId("5ee69e393260aab97ea0d58e"), title: 'Delete me' }**,  
  {
    _id: ObjectId("5ee69d943260aab97ea0d58d"),
    title: 'Thin crust pizza'
  },  
  { _id: ObjectId("5e5e9c470d33e9e8e3891b35"), title: 'Tacos' }
]

`db.examples.deleteOne({ _id: ObjectId("5ee69e393260aab97ea0d58e")})`

**Always use deleteOne, not delete, if deleting one document!!! Otherwise, it'll delete multiple.**

`cooker> db.examples.find({}, {title:1})`  
[ 
  {
    _id: ObjectId("5ee69d943260aab97ea0d58d"),
    title: 'Thin crust pizza'
  },  
  { _id: ObjectId("5e5e9c470d33e9e8e3891b35"), title: 'Tacos' }
]

# Challenges

## Challenge 2

### part 1



using **recipes** collection, return:
- top-rated recipes
- sort by most popular first
- limit to 4 results

My query: Correct!  
✅ `cooker> db.recipes.find({},{title:1, rating_avg:1}).sort({rating_avg:-1}).limit(4)`  
[
  {
    _id: ObjectId("5edf1cd43260aab97ea0d588"),
title: 'Apple Pie',
rating_avg: 4.8
},  
  {
    _id: ObjectId("5e877cba20a4f574c0aa56da"),
    title: 'Pancakes',
    rating_avg: 3.88
  },  
  {
    _id: ObjectId("5e6fd805fa98021236426a24"),
    title: 'Chicken Soft Tacos',
    rating_avg: 3.71
  },  
  {
    _id: ObjectId("5e5e9c470d33e9e8e3891b35"),
    title: 'Tacos',
    rating_avg: 3.5
  }
]

**Notes:**

It's good to precalculate fields like ratings avg, so you don't have to in the query.

### part 2

return 
- recipes tagged "Mexican"
- sort by popular first
- limit to four results

My query: Correct after 3 tries. Syntax and I used $or instead of $in
The instructor had a more concise answer though.

✅`db.recipes.find({tags:{$in:["mexican"]}}, {title:1, tags:1, rating_avg:1}).sort({rating_avg: -1}).limit(4)`  
[
  {
    _id: ObjectId("5e6fd805fa98021236426a24"),
    title: 'Chicken Soft Tacos',
    rating_avg: 3.71,
    tags: [ 'mexican', 'quick', 'easy', 'chicken' ]
  },  
  {
    _id: ObjectId("5e5e9c470d33e9e8e3891b35"),
    title: 'Tacos',
    rating_avg: 3.5,
    tags: [ 'mexican', 'quick', 'easy', 'ground beef' ]
  }
]

**Instructor's answer:**  
`cooker> db.recipes.find({tags:"mexican"}, {title:1, tags:1, rating_avg:1}).sort({rating_avg: -1}).limit(4)`

If it's seeing if one thing is in the array, you can do it this way. But if there's multiple, use the conditional operators as in my answer.

### part 3

return 
- recipes liked by userid 1
- sort by title
- no limit


My query: Correct! Changed so that I don't use the $in operator.

✅`db.recipes.find({likes:1}, {likes:1, title:1}).sort({title:1}, 1)`  
[
  {
    _id: ObjectId("5edf1cd43260aab97ea0d588"),
    title: 'Apple Pie',
    likes: [ 2, 1, 75 ]
  },  
  {
    _id: ObjectId("5e6fd805fa98021236426a24"),
    title: 'Chicken Soft Tacos',
    likes: [ 261, 1, 415 ]
  },  
  {
    _id: ObjectId("5e878f5220a4f574c0aa56db"),
    title: 'Maple Smoked Salmon',
    likes: [ 1 ]
  },  
  {
    _id: ObjectId("5e877cba20a4f574c0aa56da"),
    title: 'Pancakes',
    likes: [ 261, 415, 1, 35, 75 ]
  },  
  {
    _id: ObjectId("5e5e9c470d33e9e8e3891b35"),
    title: 'Tacos',
    likes: [ 1, 415 ]
  }
]